# Comparison of l_infty robustness curves for models trained with MMR+AT on different datasets

In [ ]:
import os
os.chdir("../")
import sys
import json
from argparse import Namespace
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import foolbox
from sklearn import metrics
from sklearn.metrics import pairwise_distances as dist
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper')

import provable_robustness_max_linear_regions.data as dt
from provable_robustness_max_linear_regions import models
from provable_robustness_max_linear_regions.models import load_model
from robustness_curves import generate_curve_data
from utils import NumpyEncoder

## Plot settings:

In [ ]:
SMALL_SIZE = 4.2
MEDIUM_SIZE = 5.8
BIGGER_SIZE = 6.0

TEXT_WIDTH = 4.8041

TICK_LABEL_TO_TICK_DISTANCE = -2  # the lower the closer

LINE_WIDTH = 0.6


def calc_fig_size(n_rows, n_cols, text_width=TEXT_WIDTH):
    ax_width = text_width / 3
    ax_height = text_width / 5
    extra_height = text_width / 4 * 2 - text_width / 5 * 2

    fig_width = n_cols * ax_width
    fig_height = n_rows * ax_height

    if fig_width > text_width:
        factor = text_width / fig_width
        fig_width *= factor
        fig_height *= factor

    fig_height += extra_height

    return fig_width, fig_height


def tex_rob(sub, sup, arg):
    return 'R_{{{}}}^{{{}}}({{{}}})'.format(sub, sup, arg)


X_EPS = r'perturbation size $\varepsilon$'
X_EPS_INF = r'$\ell_\infty$ perturbation size $\varepsilon$'
X_EPS_ONE = r'$\ell_1$ perturbation size $\varepsilon$'
X_EPS_TWO = r'$\ell_2$ perturbation size $\varepsilon$'
Y_ROB = '${}$'.format(tex_rob('', '', r'\varepsilon'))
Y_ROB_INF = '${}$'.format(tex_rob(r'\|\cdot\|_\infty', '', r'\varepsilon'))
Y_ROB_ONE = '${}$'.format(tex_rob(r'\|\cdot\|_1', '', r'\varepsilon'))
Y_ROB_TWO = '${}$'.format(tex_rob(r'\|\cdot\|_2', '', r'\varepsilon'))

# plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text', usetex=True)

colors = {
    "orange": sns.xkcd_rgb["yellowish orange"],
    "red": sns.xkcd_rgb["pale red"],
    "green": sns.xkcd_rgb["medium green"],
    "blue": sns.xkcd_rgb["denim blue"],
    "yellow": sns.xkcd_rgb["amber"],
    "purple": sns.xkcd_rgb["dusty purple"],
    "cyan": sns.xkcd_rgb["cyan"]
}

## Calculate robustness curves:
Estimated runtime (if no file with data is present): 1 day

In [ ]:
def load_from_json(file_name):

    if not os.path.exists("res/" + file_name + ".json"):
        return None
    else:
        with open("res/" + file_name + ".json", 'r') as fp:
            loaded_json =  json.load(fp)

            for key in loaded_json.keys():
                loaded_json[key]["x"] = np.array(loaded_json[key]["x"])
                loaded_json[key]["y"] = np.array(loaded_json[key]["y"])

                loaded_json[key]["y"][np.isnan(loaded_json[key]["x"])] = 1.0
                loaded_json[key]["x"] = np.nan_to_num(loaded_json[key]["x"], nan = np.nanmax(loaded_json[key]["x"]))

            return loaded_json
        
def save_to_json(dictionary, file_name):
        
    if not os.path.exists("res"):
        os.makedirs("res")

    with open("res/" + file_name + ".json", 'w') as fp:
        json.dump(dictionary, fp, cls = NumpyEncoder)


dataset_to_model_path = {"mnist": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-17 01_54_16 dataset=mnist nn_type=cnn_lenet_small p_norm=inf lmbd=0.5 gamma_rb=0.2 gamma_db=0.2 ae_frac=0.5 epoch=100.mat", 
                        "fmnist": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-16 12_08_43 dataset=fmnist nn_type=cnn_lenet_small p_norm=inf lmbd=2.0 gamma_rb=0.15 gamma_db=0.15 stage1hpl=10 ae_frac=0.5 epoch=100.mat",
                         "gts": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-17 23_43_21 dataset=gts nn_type=cnn_lenet_small p_norm=inf lmbd=1.0 gamma_rb=6.0 gamma_db=6.0 ae_frac=0.5 epoch=100.mat",
                         "cifar10": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-17 23_20_04 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.1 gamma_rb=3.0 gamma_db=3.0 ae_frac=0.5 epoch=100.mat"
                        }

n_points = 10000

robustness_curve_data = dict()

for dataset in ["mnist", "fmnist", "gts", "cifar10"]:
    
    robustness_curve_data[dataset] = load_from_json("appr_rb_curve_l_sup_model_and_dataset={}_n_points={}".format(dataset, n_points))
                                           
    if not robustness_curve_data[dataset]:
        
        _, x_test, _, y_test = dt.get_dataset(dataset)

        x_test = x_test[:n_points]
        y_test = y_test[:n_points]
        
        if dataset in ["mnist", "fmnist"]:
            x_test = x_test.reshape(n_points, 1, 28, 28, 1)
        else:
            x_test = x_test.reshape(n_points, 1, 32, 32, 3)

        model_args = Namespace()
        n_test_ex, one, model_args.height, model_args.width, model_args.n_col = x_test.shape
        model_args.n_in, model_args.n_out = model_args.height * model_args.width * model_args.n_col, y_test.shape[1]
        model_args.n_hs = []
        model_args.seed = 1
        model_args.nn_type = "cnn"
        model_args.dataset = dataset

        sess = tf.InteractiveSession()
        model, _input, _logits, _ = load_model(sess, model_args, dataset_to_model_path[dataset])
        
        f_model = foolbox.models.TensorFlowModel(_input, _logits, (0,1))
        
        args = Namespace()

        args.inputs = x_test
        args.labels = y_test
        args.f_model = f_model
        args.norms = ["inf"]
        args.save = False
        args.plot = False

        robustness_curve_data[dataset] = generate_curve_data(args)
        
        save_to_json(robustness_curve_data[dataset], "appr_rb_curve_l_sup_model_and_dataset={}_n_points={}".format(dataset, n_points))  
        
        tf.reset_default_graph()
        sess.close()   

## Plot:

In [ ]:
# name to save the plot
save_name = "fig_rc_sup_different_datasets"

# number of model types and parameter combinations
n_cols = 3
n_rows = 1

fig, ax = plt.subplots(n_rows,
                       n_cols,
                       figsize=calc_fig_size(n_rows, n_cols),
                       sharey=True)

dataset_to_color = {
    "mnist": colors["red"],
    "fmnist": colors["blue"],
    "gts": colors["green"],
    "cifar10": colors["yellow"]
}
dataset_to_label = {
    "mnist": "MNIST",
    "fmnist": "FMNIST",
    "gts": "GTS",
    "cifar10": "CIFAR10"
}

for dataset in ["mnist", "fmnist", "gts", "cifar10"]:

    dists = load_from_json(
        "appr_rb_curve_l_sup_model_and_dataset={}_n_points={}".format(dataset, n_points))

    ax[0].plot(dists["inf"]["x"],
               dists["inf"]["y"],
               c=dataset_to_color[dataset],
               label="{}".format(dataset_to_label[dataset]), linewidth=LINE_WIDTH)

for dataset in ["mnist", "fmnist"]:

    dists = load_from_json(
        "appr_rb_curve_l_sup_model_and_dataset={}_n_points={}".format(dataset, n_points))

    ax[1].plot(dists["inf"]["x"],
               dists["inf"]["y"],
               c=dataset_to_color[dataset],
               label="{}".format(dataset_to_label[dataset]), linewidth=LINE_WIDTH)

for dataset in ["gts", "cifar10"]:

    dists = load_from_json(
        "appr_rb_curve_l_sup_model_and_dataset={}_n_points={}".format(dataset, n_points))

    ax[2].xaxis.set_major_formatter(
        matplotlib.ticker.FormatStrFormatter('%.2f'))
    ax[2].plot(dists["inf"]["x"],
               dists["inf"]["y"],
               c=dataset_to_color[dataset],
               label="{}".format(dataset_to_label[dataset]), linewidth=LINE_WIDTH)

for i in range(3):
    ax[i].legend()
    ax[i].set_xlabel(X_EPS_INF)

ax[0].set_ylabel(Y_ROB_INF)

ax[2].set_xlim(-0.01, 0.2)

ax[0].tick_params(axis='both',
                  which='major',
                  pad=TICK_LABEL_TO_TICK_DISTANCE)
ax[1].tick_params(axis='both',
                  which='major',
                  pad=TICK_LABEL_TO_TICK_DISTANCE)
ax[2].tick_params(axis='both',
                  which='major',
                  pad=TICK_LABEL_TO_TICK_DISTANCE)

fig.tight_layout()
fig.savefig('res/{}.pdf'.format(save_name))